In [8]:
import os
import pandas as pd
import tqdm
import math

In [9]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Set the variables to the data folders:

In [10]:
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/MyDrive/'
data_folder = os.path.abspath(os.path.join(root_folder, 'roberta'))
model_folder = os.path.abspath(os.path.join(root_folder, 'roberta/model'))
output_folder = os.path.abspath(os.path.join(root_folder, 'roberta'))
tokenizer_folder = os.path.abspath(os.path.join(root_folder, 'roberta/tokenize'))



Load the train datafile with the product descriptions and names:

In [11]:
# Load the train dataset
train_df=pd.read_csv(datafile_path, header=0, usecols=[0,1])
# Show the count of rows
print('Num Examples: ',len(train_df))
print('Null Values\n', train_df.isna().sum())
# Drop rows with Null values
train_df.dropna(inplace=True)
print('Num Examples: ',len(train_df))

NameError: name 'datafile_path' is not defined

Then, we read the test dataset:

In [ ]:
# Load the test dataset
test_df=pd.read_csv(testfile_path, header=0)
print('Num Examples: ',len(test_df))
print('Null Values\n', test_df.isna().sum())
# there are no null values

Num Examples:  1441
Null Values
 description    0
dtype: int64


# Build a Tokenizer


In [ ]:
# Drop the files from the output dir
txt_files_dir = "./text_split"
!rm -rf {txt_files_dir}
!mkdir {txt_files_dir}

In [ ]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e)
      i+=1
    # Return the last ID
    return i


Include the training dataset to the main text file:

In [ ]:
data = train_df["description"]
# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)

31548


Also include the test dataset to the text file:

In [ ]:
data = test_df["description"]
# Removing the end of line character \n
data = data.replace("\n"," ")
print(len(data))
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

1441
32989


**Include the target variable for training** NOOOO¿?

In [ ]:
data = train_df["name"]
data = data.replace("\n"," ")
print(len(data))
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

31548
64537


In [12]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
!pip install datasets==1.0.2

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-isc7un0a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-isc7un0a
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 76, in resolve
    collected = self.factory.collect_root_requirements(root_reqs)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 538, 

In [13]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

import torch
from torch.utils.data.dataset import Dataset

In [14]:
%%time
single_txt_file = os.path.join("/content/drive/MyDrive/roberta/train_file.txt")

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training on the single file
tokenizer.train(files=[single_txt_file],
                vocab_size=8192,
                min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])


CPU times: user 1min 34s, sys: 845 ms, total: 1min 35s
Wall time: 1min 32s


In [15]:
tokenizer

Tokenizer(vocabulary_size=8192, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

The count of samples is small and the tokenizer trains very fast. Now we can save the tokenizer to disk, later we will use it to train the language model:

In [16]:
#Save the Tokenizer to disk
tokenizer.save_model(tokenizer_folder)

['/content/drive/MyDrive/roberta/tokenize/vocab.json',
 '/content/drive/MyDrive/roberta/tokenize/merges.txt']

In [17]:
# Create the tokenizer using vocab.json and mrege.txt files
tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)

In [18]:
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

Let's show some examples:

In [19]:
tokenizer.encode("The influence of K2O/(MgO+K2O) on some melt properties, including crystallization during cooling of melts and glass-forming ability, was investigated in the Li2O Al2O3 SiO2 system with low Al2O3 content. The dependence of viscosity on K2O/(MgO+K2O) above 1000 C showed a monotonic decrease due to the reduction of [MgO4] concentration and the conductivity also decreased due to the larger ion radius of K. The temperature dependence of conductivity for all melts showed an abrupt change at one temperature due to crystallization in which temperature of crystallization decreases with increase of K2O. The crystallization behavior near liquidus temperature was studied quantitatively by calculating the crystal volume fraction from apparent viscosity value. The glass-forming ability of the melts was discussed by using data related with viscosity and crystallization. Finally, it was suggested that the melts with K2O/(MgO+K2O) 0.75 have a good glass-forming ability.")

Encoding(num_tokens=236, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [20]:
# Show the tokens created
tokenizer.encode("The influence of K2O/(MgO+K2O) on some melt properties, including crystallization during cooling of melts and glass-forming ability, was investigated in the Li2O Al2O3 SiO2 system with low Al2O3 content. The dependence of viscosity on K2O/(MgO+K2O) above 1000 C showed a monotonic decrease due to the reduction of [MgO4] concentration and the conductivity also decreased due to the larger ion radius of K. The temperature dependence of conductivity for all melts showed an abrupt change at one temperature due to crystallization in which temperature of crystallization decreases with increase of K2O. The crystallization behavior near liquidus temperature was studied quantitatively by calculating the crystal volume fraction from apparent viscosity value. The glass-forming ability of the melts was discussed by using data related with viscosity and crystallization. Finally, it was suggested that the melts with K2O/(MgO+K2O) 0.75 have a good glass-forming ability.").tokens

['<s>',
 'T',
 'he',
 'Ġinfluence',
 'Ġof',
 'Ġ',
 'K',
 '2',
 'O',
 '/(',
 'M',
 'g',
 'O',
 '+',
 'K',
 '2',
 'O',
 ')',
 'Ġon',
 'Ġsome',
 'Ġmelt',
 'Ġproperties',
 ',',
 'Ġincluding',
 'Ġcrystallization',
 'Ġduring',
 'Ġcooling',
 'Ġof',
 'Ġmelts',
 'Ġand',
 'Ġglass',
 '-',
 'forming',
 'Ġability',
 ',',
 'Ġwas',
 'Ġinvestigated',
 'Ġin',
 'Ġthe',
 'Ġ',
 'L',
 'i',
 '2',
 'O',
 'Ġ',
 'A',
 'l',
 '2',
 'O',
 '3',
 'Ġ',
 'S',
 'i',
 'O',
 '2',
 'Ġsystem',
 'Ġwith',
 'Ġlow',
 'Ġ',
 'A',
 'l',
 '2',
 'O',
 '3',
 'Ġcontent',
 '.',
 'Ġ',
 'T',
 'he',
 'Ġdependence',
 'Ġof',
 'Ġviscosity',
 'Ġon',
 'Ġ',
 'K',
 '2',
 'O',
 '/(',
 'M',
 'g',
 'O',
 '+',
 'K',
 '2',
 'O',
 ')',
 'Ġabove',
 'Ġ1000',
 'Ġ',
 'C',
 'Ġshowed',
 'Ġa',
 'Ġmonot',
 'onic',
 'Ġdecrease',
 'Ġdue',
 'Ġto',
 'Ġthe',
 'Ġreduction',
 'Ġof',
 'Ġ[',
 'M',
 'g',
 'O',
 '4',
 ']',
 'Ġconcentration',
 'Ġand',
 'Ġthe',
 'Ġconductivity',
 'Ġalso',
 'Ġdecreased',
 'Ġdue',
 'Ġto',
 'Ġthe',
 'Ġlarger',
 'Ġion',
 'Ġradius',
 'Ġof',


We define some global parameters:

In [21]:
TRAIN_BATCH_SIZE = 16    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 8    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 15        # number of epochs to train (default: 10)
LEARNING_RATE = 1e-4    # learning rate (default: 0.001)
WEIGHT_DECAY = 0.01
SEED = 42               # random seed (default: 42)
MAX_LEN = 128
SUMMARY_LEN = 7

In [22]:
# Check that we have a GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [23]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

False

In [24]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [25]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [26]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

In [27]:
tokenizer

RobertaTokenizerFast(name_or_path='/content/drive/MyDrive/roberta/tokenize', vocab_size=8192, model_max_length=128, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

https://ryanong.co.uk/2020/06/11/day-163-how-to-build-a-language-model-from-scratch-implementation/

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []

        for example in df.values:
            x=tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

Concat the training and test dataset, only with the description column.

In [ ]:
# Concatenate the train dataset and the test dataset for language modelling
#df=pd.concat([train_df['description'], test_df['description']], axis=0)
#print('Total: ',len(df), len(train_df), len(test_df))


32989 31548 1441
0       towel with border with lines metallic thread .
1    printed bermuda shorts made technical fabric ....
2    bodysuit with shapewear effect . this bodysuit...
3    vneck with thin adjustable straps.height model...
4    puritan collar dress featuring long sleeves wi...
Name: description, dtype: object


Create the custom datasets, for training and evaluation:

In [ ]:
# Create the train and evaluation dataset
train_dataset = CustomDataset(train_df['description'], tokenizer)
eval_dataset = CustomDataset(test_df['description'], tokenizer)

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, text_file, tokenizer, max_lines=10):  # Added max_lines parameter
        self.examples = []

        # Read the text file and split into individual examples
        with open(text_file, 'r', encoding='utf-8') as f:
            # Initialize a line counter
            line_count = 0

            # Assuming each line is a separate example
            for line in f:
                line = line.strip()
                if line:  # Skip empty lines
                    # Tokenize the input line
                    x = tokenizer.encode_plus(
                        line,
                        max_length=MAX_LEN,
                        truncation=True,
                        padding='max_length',
                        return_tensors='pt'
                    )
                    self.examples.append(x['input_ids'].squeeze(0))  # Remove the batch dimension

                    line_count += 1  # Increment the line counter

                    # Break if the desired number of lines is reached
                    if line_count >= max_lines:
                        break

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

# Create the train and evaluation dataset, limiting to 10 lines
train_dataset = CustomDataset('/content/drive/MyDrive/roberta/train_file.txt', tokenizer, max_lines=10000)
eval_dataset = CustomDataset('/content/drive/MyDrive/roberta/val_file.txt', tokenizer, max_lines=100)

In [28]:

class CustomDataset(Dataset):
    def __init__(self, text_file, tokenizer):
        self.examples = []

        # Read the text file and split into individual examples
        with open(text_file, 'r', encoding='utf-8') as f:
            # Assuming each line is a separate example
            for line in f:
                line = line.strip()
                if line:  # Skip empty lines
                    # Tokenize the input line
                    x = tokenizer.encode_plus(
                        line,
                        max_length=MAX_LEN,
                        truncation=True,
                        padding='max_length',
                        return_tensors='pt'
                    )
                    self.examples.append(x['input_ids'].squeeze(0))  # Remove the batch dimension

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

# Create the train and evaluation dataset

train_dataset = CustomDataset('/content/drive/MyDrive/roberta/train_file.txt', tokenizer)
eval_dataset = CustomDataset('/content/drive/MyDrive/roberta/val_file.txt', tokenizer)


## Define the Data Collactor for masking our language

In [29]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Define the checkpoint directory and the map location
checkpoint_dir = '/content/drive/MyDrive/roberta/model/checkpoint-106496'  # Replace with your path
map_location = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model from the checkpoint (including safetensors)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir, device_map='auto')

# Define training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    save_total_limit=1,
)

# Create the trainer for your model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Resume training from the checkpoint
trainer.train(resume_from_checkpoint=checkpoint_dir)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/roberta/model/checkpoint-106496 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias'].
There were unexpected keys in the checkpoint model loaded: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight'].
/usr/local/lib/python3.

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

# Path to your saved checkpoint
checkpoint_dir = '/content/drive/MyDrive/roberta/checkpoint'  # Replace with the actual path
map_location = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model from the checkpoint

# Load the model checkpoint with map_location
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint_dir,
    ignore_mismatched_sizes=True,
    map_location=map_location  # This ensures the model is loaded on the correct device
)

# Define new training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    num_train_epochs=9,  # Set how many more epochs you want to train
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    save_total_limit=1,
)

# Create the Trainer, reusing the same datasets and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Resume training from the checkpoint
trainer.train(resume_from_checkpoint=checkpoint_dir, ignore_data_skip=True)


In [30]:
from transformers import Trainer, TrainingArguments
from transformers import RobertaForMaskedLM
from sklearn.metrics import precision_recall_fscore_support

print(model_folder)
checkpoint_dir = '/content/drive/MyDrive/roberta/model/checkpoint-115485'  # Replace with the actual path
model = RobertaForMaskedLM.from_pretrained(checkpoint_dir)
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=8,  # Reduce from 16 to 8
    per_device_eval_batch_size=8,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
def compute_metrics(p):
    preds = p.predictions.argmax(-1)  # Get the predicted labels
    labels = p.label_ids

    # Flatten the arrays and ignore the padding labels (-100)
    preds_flat = preds.flatten()
    labels_flat = labels.flatten()

    # Filter out the padding labels (-100)
    mask = labels_flat != -100
    preds_flat = preds_flat[mask]
    labels_flat = labels_flat[mask]

    # Compute precision, recall, F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels_flat, preds_flat, average='micro')
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(labels_flat, preds_flat, average='macro')

    return {
        'micro_f1': f1,
        'macro_f1': macro_f1,
        'precision': precision,
        'recall': recall,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall
    }

# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

    #prediction_loss_only=True,
)

/content/drive/MyDrive/roberta/model


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


And now, we are ready to train our model

In [31]:
trainer.train(resume_from_checkpoint=checkpoint_dir)

There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3262: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for 

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=115485, training_loss=0.0, metrics={'train_runtime': 0.0357, 'train_samples_per_second': 51747962.87, 'train_steps_per_second': 3234326.448, 'total_flos': 6.121378120158413e+16, 'train_loss': 0.0, 'epoch': 15.0})

In [ ]:
torch.cuda.empty_cache()


In [33]:
# Evaluate the model using the trainer
eval_results = trainer.evaluate()

print(eval_results)

KeyboardInterrupt: 

In [32]:
trainer.save_model('/content/drive/MyDrive/roberta/model')

# Checking the trained model using a Pipeline

In [33]:
from transformers import pipeline
import os
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/MyDrive/'
data_folder = os.path.abspath(os.path.join(root_folder, 'roberta'))
model_folder = os.path.abspath(os.path.join(root_folder, 'roberta/model'))
output_folder = os.path.abspath(os.path.join(root_folder, 'roberta'))
tokenizer_folder = os.path.abspath(os.path.join(root_folder, 'roberta/tokenize'))


fill_mask = pipeline(
    "fill-mask",
    model=model_folder,
    tokenizer=tokenizer_folder
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [34]:
# knit midi dress with vneckline
# =>
fill_mask("The synthesis of ceramic materials often involves <mask> processing techniques, which can influence their microstructure.")

[{'score': 0.22301055490970612,
  'token': 1666,
  'token_str': ' several',
  'sequence': 'The synthesis of ceramic materials often involves several processing techniques, which can influence their microstructure.'},
 {'score': 0.1439501941204071,
  'token': 1146,
  'token_str': ' various',
  'sequence': 'The synthesis of ceramic materials often involves various processing techniques, which can influence their microstructure.'},
 {'score': 0.08998281508684158,
  'token': 654,
  'token_str': ' different',
  'sequence': 'The synthesis of ceramic materials often involves different processing techniques, which can influence their microstructure.'},
 {'score': 0.036749836057424545,
  'token': 684,
  'token_str': ' two',
  'sequence': 'The synthesis of ceramic materials often involves two processing techniques, which can influence their microstructure.'},
 {'score': 0.033096421509981155,
  'token': 2537,
  'token_str': ' many',
  'sequence': 'The synthesis of ceramic materials often involves

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [35]:
# The test text: Round neck sweater with long sleeves
fill_mask("Round neck sweater with <mask> sleeves.")

[{'score': 0.03832438960671425,
  'token': 424,
  'token_str': ' glass',
  'sequence': 'Round neck sweater with glass sleeves.'},
 {'score': 0.026837239041924477,
  'token': 4500,
  'token_str': ' human',
  'sequence': 'Round neck sweater with human sleeves.'},
 {'score': 0.023072747513651848,
  'token': 1050,
  'token_str': ' potential',
  'sequence': 'Round neck sweater with potential sleeves.'},
 {'score': 0.021660633385181427,
  'token': 1449,
  'token_str': ' long',
  'sequence': 'Round neck sweater with long sleeves.'},
 {'score': 0.015680409967899323,
  'token': 6155,
  'token_str': ' flexible',
  'sequence': 'Round neck sweater with flexible sleeves.'}]

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

# Path to your saved checkpoint
checkpoint_dir = '/path/to/checkpoint'  # Replace with the actual path

# Load the model from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)

# Define new training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    num_train_epochs=REMAINING_EPOCHS,  # Set how many more epochs you want to train
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    save_total_limit=1,
)

# Create the Trainer, reusing the same datasets and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Resume training from the checkpoint
trainer.train(resume_from_checkpoint=checkpoint_dir)
